In [2]:
import urllib.request
import json
import pandas as pd
from datetime import datetime
import pickle
import os
from google.colab import drive

pd.set_option('display.max_columns', None)

In [3]:
import urllib.request
import json
import pandas as pd
from datetime import datetime

pd.set_option('display.max_columns', None)

YEARS_TO_FETCH = [2023, 2024, 2025]

all_sessions_df = pd.DataFrame()

print(f"Fetching session info for years: {YEARS_TO_FETCH}...")

for year in YEARS_TO_FETCH:
    print(f"--- Fetching {year} ---")
    try:
        api_url = f"https://api.openf1.org/v1/sessions?year={year}"

        with urllib.request.urlopen(api_url) as response:
            data = json.loads(response.read().decode('utf-8'))
            year_sessions_df = pd.DataFrame(data)

            all_sessions_df = pd.concat([all_sessions_df, year_sessions_df], ignore_index=True)

            print(f"Found {len(year_sessions_df)} sessions for {year}.")

    except Exception as e:
        print(f"Error fetching data for {year}: {e}")

print("\n--- All Sessions Fetched ---")

all_sessions_df['date_start'] = pd.to_datetime(all_sessions_df['date_start'])

now_utc = pd.Timestamp.now(tz='UTC')

completed_sessions_df = all_sessions_df[all_sessions_df['date_start'] < now_utc]
print(f"Total sessions found: {len(all_sessions_df)}")
print(f"Total completed sessions found: {len(completed_sessions_df)}")

print("\nSample of completed sessions:")
print(completed_sessions_df.head())

Fetching session info for years: [2023, 2024, 2025]...
--- Fetching 2023 ---
Found 113 sessions for 2023.
--- Fetching 2024 ---
Found 123 sessions for 2024.
--- Fetching 2025 ---
Found 103 sessions for 2025.

--- All Sessions Fetched ---
Total sessions found: 339
Total completed sessions found: 339

Sample of completed sessions:
   meeting_key  session_key location                date_start  \
0         1140         9222   Sakhir 2023-02-23 07:00:00+00:00   
1         1140         7763   Sakhir 2023-02-24 07:00:00+00:00   
2         1140         7764   Sakhir 2023-02-25 07:00:00+00:00   
3         1141         7765   Sakhir 2023-03-03 11:30:00+00:00   
4         1141         7766   Sakhir 2023-03-03 15:00:00+00:00   

                    date_end session_type session_name  country_key  \
0  2023-02-23T16:30:00+00:00     Practice   Practice 1           36   
1  2023-02-24T16:30:00+00:00     Practice   Practice 2           36   
2  2023-02-25T16:30:00+00:00     Practice   Practice 3     

In [4]:
import time

all_data = {}

def fetch_data(endpoint_name, session_key):
    try:
        url = f"https://api.openf1.org/v1/{endpoint_name}?session_key={session_key}"
        with urllib.request.urlopen(url) as response:
            data_json = json.loads(response.read().decode('utf-8'))
            df = pd.DataFrame(data_json)
            if not df.empty:
                print(f"  -> Success: Downloaded {len(df)} rows for {endpoint_name}.")
            else:
                print(f"  -> No data found for {endpoint_name}.")
            return df
    except Exception as e:
        print(f"  -> ERROR fetching {endpoint_name}: {e}")
        return pd.DataFrame() # Return an empty DataFrame on error

print(f"--- Starting to download CORE MODEL data for {len(completed_sessions_df)} sessions ---")

ENDPOINTS_TO_FETCH = [
    'laps',           # For tyre deg
    'stints',         # For tyre deg (compound, age)
    'weather',        # For tyre deg
    'pit',            # For tyre deg
    'session_result', # For race winner
    'starting_grid',  # For race winner
    'drivers'         # To map driver numbers to names
]

for index, session in completed_sessions_df.iterrows():
    session_key = session['session_key']
    country = session['country_name']
    year = session['year']

    print(f"\nProcessing: {year} {country} - {session['session_name']} (Key: {session_key})")

    # Create the nested dictionary for this session
    all_data[session_key] = {}

    try:
        # Loop through our defined list of endpoints
        for endpoint in ENDPOINTS_TO_FETCH:
            all_data[session_key][endpoint] = fetch_data(endpoint, session_key)
            time.sleep(0.1) # Be nice to the API, wait 100ms between calls

    except KeyboardInterrupt:
        print(f"\n--- USER STOPPED: Skipping session {session_key} and moving to the next. ---")
        continue
    except Exception as e:
        print(f"\n--- UNHANDLED ERROR: Skipping session {session_key}: {e} ---")
        continue

print("\n\n🏁 --- CORE MODEL DATA DOWNLOAD COMPLETE --- 🏁")

--- Starting to download CORE MODEL data for 339 sessions ---

Processing: 2023 Bahrain - Practice 1 (Key: 9222)
  -> Success: Downloaded 1276 rows for laps.
  -> Success: Downloaded 267 rows for stints.
  -> Success: Downloaded 590 rows for weather.
  -> No data found for pit.
  -> Success: Downloaded 19 rows for session_result.
  -> No data found for starting_grid.
  -> Success: Downloaded 19 rows for drivers.

Processing: 2023 Bahrain - Practice 2 (Key: 7763)
  -> ERROR fetching laps: HTTP Error 429: Too Many Requests
  -> Success: Downloaded 234 rows for stints.
  -> Success: Downloaded 590 rows for weather.
  -> No data found for pit.
  -> Success: Downloaded 17 rows for session_result.
  -> No data found for starting_grid.
  -> Success: Downloaded 17 rows for drivers.

Processing: 2023 Bahrain - Practice 3 (Key: 7764)
  -> Success: Downloaded 1399 rows for laps.
  -> Success: Downloaded 271 rows for stints.
  -> Success: Downloaded 581 rows for weather.
  -> No data found for pit

In [9]:
import pandas as pd
from google.colab import files

print("--- Processing all downloaded data... ---")
print("Combining all downloaded data into master tables...")
try:
    all_laps_dfs = [d['laps'] for d in all_data.values() if d.get('laps') is not None and not d['laps'].empty]
    all_stints_dfs = [d['stints'] for d in all_data.values() if d.get('stints') is not None and not d['stints'].empty]
    all_weather_dfs = [d['weather'] for d in all_data.values() if d.get('weather') is not None and not d['weather'].empty]
    all_pit_dfs = [d['pit'] for d in all_data.values() if d.get('pit') is not None and not d['pit'].empty]

    all_laps_df = pd.concat(all_laps_dfs, ignore_index=True)
    all_stints_df = pd.concat(all_stints_dfs, ignore_index=True)
    all_weather_df = pd.concat(all_weather_dfs, ignore_index=True)
    all_pit_df = pd.concat(all_pit_dfs, ignore_index=True)

except Exception as e:
    print(f"ERROR: Failed to combine initial data. {e}")
#BUILD CSV of TYRE DEGRADATION ---
print("\n--- Building Tyre Degradation CSV (tyre_degradation.csv) ---")
try:
    session_info_df = completed_sessions_df[['session_key', 'country_name', 'session_name']]
    laps_merged_df = pd.merge(all_laps_df, session_info_df, on='session_key', how='left')
    print("Merging stint data...")
    all_stints_df['lap_end'] = pd.to_numeric(all_stints_df['lap_end'], errors='coerce')
    all_stints_df['lap_start'] = pd.to_numeric(all_stints_df['lap_start'], errors='coerce')
    all_stints_df.dropna(subset=['lap_end', 'lap_start'], inplace=True)

    stint_lap_count = (all_stints_df['lap_end'] - all_stints_df['lap_start'] + 1).astype(int)
    valid_stints_df = all_stints_df[stint_lap_count > 0].copy()
    valid_stint_lap_count = stint_lap_count[stint_lap_count > 0]

    stints_expanded = valid_stints_df.loc[valid_stints_df.index.repeat(valid_stint_lap_count)]
    stints_expanded['lap_number'] = stints_expanded.groupby(['session_key', 'driver_number', 'stint_number']).cumcount() + stints_expanded['lap_start']

    laps_merged_df = pd.merge(
        laps_merged_df,
        stints_expanded[['session_key', 'driver_number', 'lap_number', 'compound', 'tyre_age_at_start']],
        on=['session_key', 'driver_number', 'lap_number'],
        how='left'
    )
    laps_merged_df['tyre_age'] = (laps_merged_df['lap_number'] - laps_merged_df.groupby(['session_key', 'driver_number', 'compound'])['lap_number'].transform('min') + laps_merged_df['tyre_age_at_start'])

    print("Merging pit data...")
    if not all_pit_df.empty:
        laps_merged_df = pd.merge(
            laps_merged_df,
            all_pit_df[['session_key', 'driver_number', 'lap_number', 'pit_duration']],
            on=['session_key', 'driver_number', 'lap_number'],
            how='left'
        )
    else:
        laps_merged_df['pit_duration'] = pd.NA

    print("Merging weather data...")
    if not all_weather_df.empty:
        laps_merged_df.dropna(subset=['date_start'], inplace=True)
        laps_merged_df['date_start'] = pd.to_datetime(laps_merged_df['date_start'], format='mixed')
        all_weather_df['date'] = pd.to_datetime(all_weather_df['date'], format='mixed')

        laps_merged_df = laps_merged_df.sort_values(by='date_start')
        all_weather_df = all_weather_df.sort_values(by='date')

        laps_merged_df = pd.merge_asof(
            laps_merged_df,
            all_weather_df[['date', 'session_key', 'air_temperature', 'track_temperature', 'rainfall', 'wind_speed']],
            left_on='date_start',
            right_on='date',
            by='session_key',
            direction='nearest'
        )
    else:
        print("No weather data found, skipping weather merge.")
        laps_merged_df['air_temperature'] = pd.NA
        laps_merged_df['track_temperature'] = pd.NA
        laps_merged_df['rainfall'] = pd.NA
        laps_merged_df['wind_speed'] = pd.NA

    tyre_deg_columns = [
        'country_name', 'session_name', 'date_start', 'driver_number', 'lap_number',
        'duration_sector_1', 'duration_sector_2', 'duration_sector_3',
        'i1_speed', 'i2_speed', 'st_speed', 'is_pit_out_lap', 'lap_duration',
        'segments_sector_1', 'segments_sector_2', 'segments_sector_3',
        'compound', 'tyre_age', 'pit_duration',
        'air_temperature', 'track_temperature', 'rainfall', 'wind_speed',
        'session_key', 'meeting_key' # Keep keys for reference
    ]
    final_tyre_deg_cols = [col for col in tyre_deg_columns if col in laps_merged_df.columns]
    tyre_deg_final_df = laps_merged_df[final_tyre_deg_cols]

    print("Saving tyre_degradation.csv...")
    tyre_deg_final_df.to_csv('tyre_degradation.csv', index=False)
    files.download('tyre_degradation.csv')
    print("-> Download triggered for tyre_degradation.csv")

except Exception as e:
    print(f"ERROR building tyre degradation CSV: {e} ---")


print("\n\nTyre degradation CSV processing complete!")

--- Processing all downloaded data... ---
Combining all downloaded data into master tables...

--- Building Tyre Degradation CSV (tyre_degradation.csv) ---


/tmp/ipython-input-2161224853.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_pit_df = pd.concat(all_pit_dfs, ignore_index=True)


Merging stint data...
Merging pit data...
Merging weather data...
Saving tyre_degradation.csv...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

-> Download triggered for tyre_degradation.csv


✅ --- Tyre degradation CSV processing complete! ---
